In [2]:
import numpy as np
from keras.datasets import mnist

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape

(60000, 28, 28)

In [5]:
images = x_train[:10000].reshape(10000, 28*28)
labels = y_train[0:10000].reshape(10000,1)

In [6]:
def preparing_data(X):
    mean_x = np.mean(X)
    std = np.sum(X**2)/(X.shape[0]-1)
    return (X - mean_x)/std

In [7]:
images = preparing_data(images)

In [8]:
from neural_network import NeuralNetwork

In [23]:
def fit(nn:NeuralNetwork, X_train, y_train, epochs, batch_size, learing_rate):
    loss = 0
    for it in range(epochs):
        batch_indexes = np.random.randint(0, X_train.shape[0], batch_size)
        batch_X = X_train[batch_indexes]
        batch_y = y_train[batch_indexes]
        output = nn.forward(batch_X)
        prediction = nn.softmax(output)
        loss = nn.CrossEntropyLoss(prediction, batch_y)
        nn.backward(learing_rate)
    return loss

In [24]:
count_of_features = images.shape[1]
count_classes = 10

In [38]:
params = {'epochs':range(100,501,100), 'batch_size':range(100,501,100)}
learing_rate = 1e-2

In [45]:
max_acc = 0
best_params = {'epochs':0, 'batch_size':0}
for epochs in params['epochs']:
    for batch_size in params['batch_size']:
        
        nn = NeuralNetwork()
        nn.CreateLayerRelu((count_of_features, 1000))
        nn.CreateLayerRelu((1000, 500))
        nn.CreateLayerLinear((500, count_classes))

        val_indexes = np.random.randint(0,10000,1000)
        
        X_val = images[val_indexes]
        y_val = labels[val_indexes]
        
        X_train = np.delete(images, val_indexes, axis=0)
        y_train = np.delete(labels, val_indexes, axis=0)
        
        loss = fit(nn, X_train, y_train, epochs, batch_size, learing_rate)
        output = nn.forward(X_val)
        prediction = nn.softmax(output)
        prediction = np.argmax(prediction, axis=1).reshape(-1,1)
        count_of_rigth = np.sum(prediction == y_val)
        accuracy = count_of_rigth / y_val.shape[0]
        if accuracy > max_acc:
            max_acc = accuracy
            best_params['epochs'] = epochs
            best_params['batch_size'] = batch_size
        print('epochs:', epochs, 'batch_size:', batch_size, 'loss', loss, 'accuracy:', accuracy)

epochs: 100 batch_size: 100 loss 0.2766492063386447 accuracy: 0.895
epochs: 100 batch_size: 200 loss 0.21612530298869156 accuracy: 0.9
epochs: 100 batch_size: 300 loss 0.10238691694076278 accuracy: 0.916
epochs: 100 batch_size: 400 loss nan accuracy: 0.101
epochs: 100 batch_size: 500 loss 0.1304527662667365 accuracy: 0.908
epochs: 200 batch_size: 100 loss 0.1492115994842655 accuracy: 0.915
epochs: 200 batch_size: 200 loss 0.028210187557656124 accuracy: 0.924
epochs: 200 batch_size: 300 loss 0.07226845537367318 accuracy: 0.934
epochs: 200 batch_size: 400 loss 0.02250999983010468 accuracy: 0.919


C:\Users\zanzi\Desktop\Python\Mnist_training\neural_network.py:23: RuntimeWarning: invalid value encountered in multiply
  grad_X = (output_grad*(self.HeavySide(self.output))).dot(self.W.T)
C:\Users\zanzi\Desktop\Python\Mnist_training\neural_network.py:24: RuntimeWarning: invalid value encountered in multiply
  grad_W = self.X.T.dot(output_grad * self.HeavySide(self.output))


epochs: 200 batch_size: 500 loss nan accuracy: 0.101
epochs: 300 batch_size: 100 loss 0.03946656296688626 accuracy: 0.92
epochs: 300 batch_size: 200 loss 0.0408268299669178 accuracy: 0.92
epochs: 300 batch_size: 300 loss 0.018476197209085245 accuracy: 0.939
epochs: 300 batch_size: 400 loss 0.023920076327174106 accuracy: 0.937
epochs: 300 batch_size: 500 loss 0.011068029986263692 accuracy: 0.926
epochs: 400 batch_size: 100 loss 0.025151734753932457 accuracy: 0.91
epochs: 400 batch_size: 200 loss 0.013906880347171031 accuracy: 0.933
epochs: 400 batch_size: 300 loss 0.012741605779973934 accuracy: 0.931
epochs: 400 batch_size: 400 loss 0.01219096535276263 accuracy: 0.917
epochs: 400 batch_size: 500 loss 0.010177481861426642 accuracy: 0.921
epochs: 500 batch_size: 100 loss 0.01355653067650948 accuracy: 0.918
epochs: 500 batch_size: 200 loss 0.01121183603313293 accuracy: 0.923
epochs: 500 batch_size: 300 loss 0.009893622122045753 accuracy: 0.935
epochs: 500 batch_size: 400 loss 0.00661766445

In [49]:
best_params['learing_rate'] = learing_rate

In [53]:
nn = NeuralNetwork()
nn.CreateLayerRelu((count_of_features, 1000))
nn.CreateLayerRelu((1000, 500))
nn.CreateLayerLinear((500, count_classes))
fit(nn, images, labels, **best_params)

0.02782727386229502

In [54]:
x_test = x_test[:1000].reshape(-1, 28*28)
x_test = preparing_data(x_test)

In [55]:
y_test = y_test[:1000].reshape(-1,1)

In [56]:
output = nn.forward(x_test)
prediction = nn.softmax(output)

In [57]:
prediction = np.argmax(prediction, axis=1)

In [58]:
prediction = prediction.reshape(-1,1)

In [59]:
count_of_rigth = np.sum(prediction == y_test)

In [60]:
accuracy = count_of_rigth / y_test.shape[0]
accuracy

0.902

Получили точность определения правильной цифры 90.2% на тестовых данных при параметрах которые показали лучшую точность на валидационных данных